# Auteurs en graphe
## Idée 1 :  association par période
On associe un lien entre deux auteurs si ils ont utilisé le mot tous les deux la même semaine.
### Justification
Cela pourrait montrer une proximité de difffusion...
### Algorithme
* Choisir un mot (ultérieurement : détection automatique, choisir plusieurs mots)
* Rassembler toutes les lettres l'utilisant 
* Classer les lettres par semaine
* Ajouter dans un graphe d'adjacence tous les auteurs utilisant ce mot

### Résultat / Affichage
* Sous forme de graphe avec le nom du groupe ?

### Calculs supplémentaires
* Observer la présence des groupes

### Améliorations possibles
* Par période plutôt que la même semaine

### Continuité
* Observer si il y a une sorte de diffusion à travers le graphe au cours du temps

In [1]:
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt
data_path = "../data/"

In [2]:
# Importation des données utilisées
df = pd.read_pickle(data_path+"df_2017_avec_auteurs.pkl")
ref_noms = df[['nom_auteur','ref_auteur','groupe_auteur']].groupby('ref_auteur').first()

In [4]:
# On choisit d'abord une taille maximale du nombre de lettres pour les tests, mettre bcp pour sans limite
MAX_LEN = 1000

# On choisit le titre ou la question ou les deux
titre = True
question = True

# Choix du mot, ou une expression Regex arbitraire pour l'instant
reg_exp = 'racisme'

In [5]:
# Construction de la liste des supports
support = []
if titre :
    support.append('titre')
if question :
    support.append('question')

In [6]:
# On rassemble toutes les lettres utilisant le mot
df_filtered = df[df[support].sum(axis=1).str.contains(reg_exp,regex=True)].reset_index().head(MAX_LEN)
print(len(df_filtered), "lettres contiennent l'expression ", reg_exp)

45 lettres contiennent l'expression  racisme


In [7]:
# On sélectionne tous les auteurs contenus dans les lettres
auteurs_filtered = df_filtered[['nom_auteur','ref_auteur','groupe_auteur']].groupby('ref_auteur').first()
auteurs_filtered['nombre_lettres'] = df_filtered.groupby('ref_auteur').count()['question']
print(len(auteurs_filtered), "auteurs ont écrit une lettre contenant l'expression ", reg_exp)

32 auteurs ont écrit une lettre contenant l'expression  racisme


In [8]:
# Regrouper les lettres par semaine
print(len(df_filtered.groupby('date_question').count()),"semaines différentes")

34 semaines différentes


In [9]:
# matrice d'adjacence
mat_adj = pd.DataFrame(columns = auteurs_filtered.index, index = auteurs_filtered.index).fillna(0)

# Ajouter un lien à la matrice d'adjacence à chaque fois que deux auteurs utilisent le mot la même semaine

# Fonction qui parcours une liste d'auteurs et complète la matrice
# On fait 'set' pour éviter qu'un auteur ayant un lien de 2 aie 2 liens avec tout le monde
def add_auteurs(x):
    for a1 in set(x['ref_auteur']):    
        for a2 in set(x['ref_auteur']):
            mat_adj.loc[a1,a2] +=1
    
# A chaque semaine, on applique la fonction add_auteurs sur la liste des auteurs de la semaine
df_filtered.groupby('date_question').apply(add_auteurs)
mat_adj.head(10)

ref_auteur,PA267901,PA345722,PA429893,PA606212,PA610681,PA643089,PA712014,PA717379,PA718744,PA718784,...,PA720846,PA720870,PA721226,PA721270,PA721278,PA721876,PA721932,PA721946,PA722062,PA722292
ref_auteur,,,,,,,,,,,,,,,,,,,,,
PA267901,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
PA345722,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
PA429893,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
PA606212,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
PA610681,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
PA643089,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
PA712014,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
PA717379,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
PA718744,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [15]:
# Taille minimale des liens à afficher, nécessaire pour ne pas que ce qoit illisible et très long
MIN_SIZE = 0

# Pourcentage minimal des liens à afficher, nécessaire pour ne pas que ce qoit illisible et très long
MIN_PCT = 0.1

# Titre du graphe
title  = "Liens des utilisations sur la même semaine de l'expresssion {} pour les liens de plus de {} occurences.".format(reg_exp,MIN_SIZE)

In [16]:
# Création du graphe en .dot
fdot = open("graph.dot", "w")
fdot.write('graph{\n') # graphe non orienté
fdot.write('label="{}";\n'.format(title)) # Nom du graphe
fdot.write('labelloc="t";\n') # Localisation des labels

cpt = 0 # Compteur pour connaître l'avancement
cpt_l = 0 # Compteur des liens 
nb = len(mat_adj.columns) # Nombre total d'auteurs

# On parcours les auteurs
for i in range(nb):
    
    # Pour connaître l'avancement
    if i%(nb//10) == 0 :
        print("{}% ... ".format(round(100*cpt/nb)),end = '')
    
    a1 = mat_adj.columns[i] # Auteur 1
    nbl_1 = auteurs_filtered.loc[a1,'nombre_lettres'] # Nombre de lettres envoyées par a1
    
    # On parcours les auteurs
    for j in range(i+1,nb):
        a2 = mat_adj.columns[j]  # Auteur 2
        nbl_2 = auteurs_filtered.loc[a2,'nombre_lettres'] # Nombre de lettres envoyées par a2
        liens = (mat_adj.loc[a1,a2])
        pct_1 = liens/nbl_1 # Proportion des liens avec cet auteur
        pct_2 = liens/nbl_2 # Proportion des liens avec cet auteur 
            
        if mat_adj.loc[a1,a2]>=MIN_SIZE and (pct_1 > MIN_PCT or pct_2 > MIN_PCT):
            cpt_l+=1
            ea1 = auteurs_filtered.loc[a1]
            na1 = "{} ({}) {}".format(ea1['nom_auteur'],ea1['groupe_auteur'],nbl_1) # Nom de l'auteur
            ea2 = auteurs_filtered.loc[a2]
            na2 = "{} ({}) {}".format(ea2['nom_auteur'],ea2['groupe_auteur'],nbl_2) # Nom de l'auteur
            fdot.write('"{}"--"{}"[label="{} ({:0.2f} - {:0.2f})"]\n'.format(na1,na2,liens,pct_1, pct_2))
    cpt+=1

fdot.write('}') # Fin
fdot.close()
print("\nIl y a au total", cpt_l, "liens")

0% ... 9% ... 19% ... 28% ... 38% ... 47% ... 56% ... 66% ... 75% ... 84% ... 94% ... 
Il y a au total 13 liens


In [17]:
# Créer le graph ps
!dot -Tps graph.dot -o graph.ps

## Résultats
### reg_exp = *racisme*, titre = **True**, question = **True**, MIN_SIZE = **0**, MIN_PCT = **0.0**, pas de MAX_LEN
* **45** lettres
* **32** auteurs
* **34** semaines
* **13** liens
* Calcul rapide
* Graphe lisible
* grosse proximité entre les partis

### reg_exp = *covid*, titre = **True**, question = **True**, MIN_SIZE = **5**, MIN_PCT = **0.1**, pas de MAX_LEN
* **953** lettres
* **240** auteurs
* **10** semaines
* **141** liens
* Calcul lent
* Graphe illisible